<a href="https://colab.research.google.com/github/adrianograms/Movie_Rating_Recommendation/blob/master/Netflix_Rating_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

For this notebook we will need pyspark to manipulate the dataset we are using. Our dataset have a load of register (500mb), so instead of pandas, we will need spark to properly explore this data

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 49.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ca614ff328371c2dbcf94ba8cecf002f61a16412707162550f0d58846262a8dd
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# unzip the dataset
!unzip '/content/drive/MyDrive/Colab Notebooks/Kaggle/Netflix Movie Rating/data/datasets.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/Kaggle/Netflix Movie Rating/data/datasets.zip
  inflating: Netflix_Dataset_Movie.csv  
  inflating: Netflix_Dataset_Rating.csv  


In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[*]") \
      .appName("API") \
      .getOrCreate() 

# EDA (Exploratory Data Analysis)

In [4]:
df_ratings = spark.read.csv('Netflix_Dataset_Rating.csv',header=True, inferSchema=True)
df_ratings.show(5)

+-------+------+--------+
|User_ID|Rating|Movie_ID|
+-------+------+--------+
| 712664|     5|       3|
|1331154|     4|       3|
|2632461|     3|       3|
|  44937|     5|       3|
| 656399|     4|       3|
+-------+------+--------+
only showing top 5 rows



Here we have our dataset. There's only 3 columns, with them being, User_ID, Rating and Movie_ID, basicacly all the information we need to do a collaborative filtering to recommend movies based on people with similar taste

In [5]:
df_ratings.count()

17337458

In [6]:
df_ratings.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Movie_ID: integer (nullable = true)



In [7]:
df_ratings.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in df_ratings.columns]).show()

+-------+------+--------+
|User_ID|Rating|Movie_ID|
+-------+------+--------+
|      0|     0|       0|
+-------+------+--------+



In [8]:
df_movies = spark.read.csv('Netflix_Dataset_Movie.csv', header=True, inferSchema=True)
df_movies.show(5)

+--------+----+--------------------+
|Movie_ID|Year|                Name|
+--------+----+--------------------+
|       1|2003|     Dinosaur Planet|
|       2|2004|Isle of Man TT 20...|
|       3|1997|           Character|
|       4|1994|Paula Abdul's Get...|
|       5|2004|The Rise and Fall...|
+--------+----+--------------------+
only showing top 5 rows



Here we have the dataset with all the information about the movies. There's not much information to use, only the ID, year and name, so no recommendation based on genre, budget, or anything like that

In [9]:
df_movies.count()

17770

In [10]:
df_raings_movies = df_movies.join(df_ratings, 'Movie_ID', how='inner')
df_raings_movies.show(5)

+--------+----+---------+-------+------+
|Movie_ID|Year|     Name|User_ID|Rating|
+--------+----+---------+-------+------+
|       3|1997|Character| 712664|     5|
|       3|1997|Character|1331154|     4|
|       3|1997|Character|2632461|     3|
|       3|1997|Character|  44937|     5|
|       3|1997|Character| 656399|     4|
+--------+----+---------+-------+------+
only showing top 5 rows



In [11]:
df_raings_movies_summary = df_raings_movies.groupBy('Movie_ID', 'Year', 'Name')\
  .agg(
      f.mean('Rating').alias('Rating_Mean'), 
      f.count('User_ID').alias('Count_Reviews'))

df_raings_movies_summary.show(5)

+--------+----+------------------+------------------+-------------+
|Movie_ID|Year|              Name|       Rating_Mean|Count_Reviews|
+--------+----+------------------+------------------+-------------+
|     582|1986|      Jo Jo Dancer| 2.961183891314896|         2061|
|     181|2004|     The Last Shot| 2.604270462633452|         2810|
|    1022|1993|       The Snapper|3.2859654263111633|         3413|
|    1646|1983|     The Dead Zone|3.6435392935220343|         8691|
|     156|1992|Husbands and Wives| 3.463768115942029|         7107|
+--------+----+------------------+------------------+-------------+
only showing top 5 rows



This dataframe have the summarized information about the movies and their ratings, like rating mean and number of reviews

In [12]:
# Movies with the highest ratings
df_raings_movies_summary.orderBy('Rating_Mean',ascending=False).show(5, False)

+--------+----+-------------------------------------+-----------------+-------------+
|Movie_ID|Year|Name                                 |Rating_Mean      |Count_Reviews|
+--------+----+-------------------------------------+-----------------+-------------+
|3456    |2004|Lost: Season 1                       |4.665432098765432|4860         |
|2102    |1994|The Simpsons: Season 6               |4.589824034920202|7331         |
|3444    |2004|Family Guy: Freakin' Sweet Collection|4.520766378244747|8090         |
|1476    |2004|Six Feet Under: Season 4             |4.461601211979955|8581         |
|4238    |2000|Inu-Yasha                            |4.457773512476008|1042         |
+--------+----+-------------------------------------+-----------------+-------------+
only showing top 5 rows



In [13]:
# Movies with more reviews
df_raings_movies_summary.orderBy('Count_Reviews',ascending=False).show(5, False)

+--------+----+------------------------------------------------------+------------------+-------------+
|Movie_ID|Year|Name                                                  |Rating_Mean       |Count_Reviews|
+--------+----+------------------------------------------------------+------------------+-------------+
|1905    |2003|Pirates of the Caribbean: The Curse of the Black Pearl|4.146333546871664 |117075       |
|2452    |2001|Lord of the Rings: The Fellowship of the Ring         |4.43148917942777  |102721       |
|4306    |1999|The Sixth Sense                                       |4.3114694850355555|102376       |
|571     |1999|American Beauty                                       |3.937732873336619 |101450       |
|3860    |2003|Bruce Almighty                                        |3.4043228981683495|98545        |
+--------+----+------------------------------------------------------+------------------+-------------+
only showing top 5 rows



In [14]:
# Best movies by year
df_raings_movies_summary\
  .groupBy('Year').agg(\
                       f.max('Rating_Mean').alias('Biggest_Rating_Mean'),
                       f.max_by('Name', 'Rating_Mean').alias('Biggest_Rating_Name'),
                       f.max_by('Count_Reviews', 'Rating_Mean').alias('Biggest_Rating_Count_Reviews'),
                       f.max('Count_Reviews').alias('Biggest_Count_Reviews'),
                       f.max_by('Name', 'Count_Reviews').alias('Biggest_Count_Reviews_Name'),
                       f.max_by('Rating_Mean', 'Count_Reviews').alias('Biggest_Count_Rating_Mean'))\
  .orderBy('Year', ascending=False).show(5, False)

+----+-------------------+-------------------------+----------------------------+---------------------+------------------------------------------------------+-------------------------+
|Year|Biggest_Rating_Mean|Biggest_Rating_Name      |Biggest_Rating_Count_Reviews|Biggest_Count_Reviews|Biggest_Count_Reviews_Name                            |Biggest_Count_Rating_Mean|
+----+-------------------+-------------------------+----------------------------+---------------------+------------------------------------------------------+-------------------------+
|2005|4.244315458757122  |Batman Begins            |37558                       |37558                |Batman Begins                                         |4.244315458757122        |
|2004|4.665432098765432  |Lost: Season 1           |4860                        |92893                |Shrek 2                                               |4.121559213288407        |
|2003|4.395903857377832  |Finding Nemo (Widescreen)|94235                  

In [15]:
# Summary of the dataframe
df_raings_movies_summary.select('Year', 'Rating_Mean', 'Count_Reviews').summary().show()

+-------+------------------+------------------+------------------+
|summary|              Year|       Rating_Mean|     Count_Reviews|
+-------+------------------+------------------+------------------+
|  count|              1350|              1350|              1350|
|   mean|           1990.38| 3.485986436639173|12842.561481481482|
| stddev|15.427057858147705|0.4225944485920032| 17805.33471936964|
|    min|              1920|1.9620309050772626|              1042|
|    25%|              1985|3.2098535286284955|              2607|
|    50%|              1996|3.5085256562772624|              5222|
|    75%|              2001| 3.768066707844348|             14808|
|    max|              2005| 4.665432098765432|            117075|
+-------+------------------+------------------+------------------+



In [16]:
# Information about the users (number of reviews by user and mean of their ratings)
df_user_info =  df_ratings.groupBy('User_ID').agg(f.count('User_ID').alias('Count_reviews'), f.mean('Rating').alias('Rating_Mean'))
df_user_info.show(5)

+-------+-------------+------------------+
|User_ID|Count_reviews|       Rating_Mean|
+-------+-------------+------------------+
|2427438|           68| 4.117647058823529|
|2496050|          217|3.1705069124423964|
|1377693|          218|3.5275229357798166|
|2190625|          298| 3.151006711409396|
|1560852|           68| 4.014705882352941|
+-------+-------------+------------------+
only showing top 5 rows



In [17]:
# Summary of the users info
df_user_info.select('Count_reviews', 'Rating_Mean').summary().show()

+-------+------------------+-------------------+
|summary|     Count_reviews|        Rating_Mean|
+-------+------------------+-------------------+
|  count|            143458|             143458|
|   mean|120.85389451965035|  3.636473203266291|
| stddev| 79.78370234809589|0.41403265068894873|
|    min|                 5|                1.0|
|    25%|                67| 3.3676470588235294|
|    50%|                95| 3.6315789473684212|
|    75%|               147| 3.9038461538461537|
|    max|              1344|                5.0|
+-------+------------------+-------------------+



# Collaborative Filtering

In [18]:
from pyspark.ml.recommendation import ALS

# Create ALS model
als = ALS(
         userCol="User_ID", 
         itemCol="Movie_ID",
         ratingCol="Rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)

model_als = als.fit(df_ratings)

In [19]:
def getNameMovie(id_movie, df_movies):
  """Get the name of the movie based on the id

  Args:
      id_movie (int): id movie
      df_movies (dataframe): dataframe with all the movies (name and id)

  Returns:
      string: name of the movie
  """
  try:
    return df_movies.where(f'Movie_ID = {id_movie}').collect()[0]['Name']
  except IndexError:
    return 'Movie Not Found'

def movieRecommendations(user_id, number_recommendations, df_ratings, df_movies, model_als):
  """Recommend a X number of movies to a specific user

  Args:
      user_id (int): User id
      number_recommendations (int): Number of recommendations
      df_ratings (dataframe): dataframe with the ratings
      df_movies (dataframe): datagrame with the movies (name and id)
      model_als (model): model trained to recommend the movies =

  Returns:
      dict: return all the movies recommended (name and id)
  """
  df_user_id = spark.createDataFrame([{'User_ID': user_id}])
  recommendations = model_als.recommendForUserSubset(df_user_id ,number_recommendations)

  if not len(recommendations.collect()):
    raise ValueError('User not found')

  recommendations_movies = []
  for recommendation in recommendations.collect()[0]['recommendations']:
    recommendation_movies = {}
    recommendation_movies['Movie_ID'] = recommendation['Movie_ID']
    recommendation_movies['Movie_Name'] = getNameMovie(recommendation['Movie_ID'], df_movies)
    recommendations_movies.append(recommendation_movies)
  return recommendations_movies

In [20]:
# Test (User that exists)
movieRecommendations(712664, 10, df_ratings, df_movies, model_als)

[{'Movie_ID': 2102, 'Movie_Name': 'The Simpsons: Season 6'},
 {'Movie_ID': 3290, 'Movie_Name': 'The Godfather'},
 {'Movie_ID': 2172, 'Movie_Name': 'The Simpsons: Season 3'},
 {'Movie_ID': 4353, 'Movie_Name': 'Curb Your Enthusiasm: Season 3'},
 {'Movie_ID': 872, 'Movie_Name': 'Seven Samurai'},
 {'Movie_ID': 3046, 'Movie_Name': 'The Simpsons: Treehouse of Horror'},
 {'Movie_ID': 722, 'Movie_Name': 'The Wire: Season 1'},
 {'Movie_ID': 3456, 'Movie_Name': 'Lost: Season 1'},
 {'Movie_ID': 3444, 'Movie_Name': "Family Guy: Freakin' Sweet Collection"},
 {'Movie_ID': 4115, 'Movie_Name': 'The Simpsons: Bart Wars'}]

In [21]:
# Test (User that not exists)
movieRecommendations(1, 10, df_ratings, df_movies, model_als)

ValueError: ignored